In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [ ]:
# Load the dataset
df = pd.read_csv('movies.csv')

In [ ]:
df.head()

,Unnamed: 0,id,title,overview,release_date,popularity,vote_average,vote_count
0,0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,84.330,8.7,21840
1,1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",1995-10-19,28.240,8.7,3726
2,2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,84.535,8.7,16263
3,3,424,Schindler's List,The true story of how businessman Oskar Schind...,1993-12-15,49.037,8.6,12947
4,4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...,1974-12-20,61.377,8.6,9799


In [ ]:
# Select just relevant features
relevant_features = ['title', 'overview', 'release_date', 'runtime', 'vote_average', 'vote_count']
relevant_features.remove('runtime')
df = df[relevant_features]

In [ ]:
pwd

'/content'

In [ ]:
# Convert release_date into pandas datetime format
df['release_date'] = pd.to_datetime(df['release_date'], errors = 'coerce')

# Extract year from release_date-column and store the values into a new year-column
df['year'] = pd.DatetimeIndex(df['release_date']).year

# Helper function to convert NaN to 0, if there are any, and all other years to integers.
def convert_int(x):
   try:
       return int(x)
   except:
       return 0

<ipython-input-6-38fda4e7fb35>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_date'] = pd.to_datetime(df['release_date'], errors = 'coerce')
<ipython-input-6-38fda4e7fb35>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = pd.DatetimeIndex(df['release_date']).year


In [ ]:
# Apply convert_int to the year feature
df['year'] = df['year'].apply(convert_int)

# Drop the release_date column
df = df.drop('release_date', axis = 1)

<ipython-input-7-b95844d7ec32>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['year'].apply(convert_int)


In [ ]:
# Convert all NaN into stringified empty lists
df['overview'] = df['overview'].fillna('[]')

In [ ]:
# # Apply literal_eval to convert stringified empty lists to the list object
# df['overview'] = df['overview'].apply(literal_eval)

In [ ]:
# Convert list of dictionaries to a list of strings
df['overview'] = df['overview'].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])

In [ ]:
# Create a new feature by exploding genres
s = df.apply(lambda x: pd.Series(x['overview']),axis = 1).stack().reset_index(level = 1, drop = True)

# Name the new feature as 'genre'
s.name = 'overview'

# Create a new dataframe gen_df which by dropping the old 'genres' feature and adding the new 'genre'.
gen_df = df.drop('overview', axis = 1).join(s)

# Print the head of the new gen_df
gen_df.head()

<ipython-input-11-dd7866e4bd12>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df.apply(lambda x: pd.Series(x['overview']),axis = 1).stack().reset_index(level = 1, drop = True)


,title,vote_average,vote_count,year,overview
0,The Shawshank Redemption,8.7,21840,1994,NaN
1,Dilwale Dulhania Le Jayenge,8.7,3726,1995,NaN
2,The Godfather,8.7,16263,1972,NaN
3,Schindler's List,8.6,12947,1993,NaN
4,The Godfather: Part II,8.6,9799,1974,NaN


In [ ]:
def build_chart(gen_df, percentile = 0.8):
   # Ask for preferred genres
   print("Input preferred overview")
   overview = input()
   print(overview)

   # Ask for lower limit of duration
   print("Input shortest duration")
   low_time = int(input())
   print(low_time)

   # Ask for upper limit of duration
   print("Input longest duration")
   high_time = int(input())
   print(high_time)

   # Ask for lower limit of timeline
   print("Input earliest year")
   low_year = int(input())
   print(low_year)

   # Ask for upper limit of timeline
   print("Input latest year")
   high_year = int(input())
   print(high_year)

   # Define a new movies variable to store the preferred movies. Copy the contents of gen_df to movies
   movies = gen_df.copy()

   if 'runtime' in gen_df.columns:
    # Filter based on the condition
    movies = movies[(movies['overview'] == overview) &
                    (movies['runtime'] >= low_time) &
                    (movies['runtime'] <= high_time) &
                    (movies['year'] >= low_year) &
                    (movies['year'] <= high_year)]
  #  else:
  #     # Handle the case where the column doesn't exist
  #     print("")

   # Compute the values of C and m for the filtered movies
   C = movies['vote_average'].mean()
   m = movies['vote_count'].quantile(percentile)

   # Only consider movies that have higher than m votes. Save this in a new dataframe q_movies
   q_movies = movies.copy().loc[movies['vote_count'] >= m]

   # Calculate score using the IMDB formula
   q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average'])
                                      + (m/(m+x['vote_count']) * C), axis = 1)

   # Sort movies in descending order of their scores
   q_movies = q_movies.sort_values('score', ascending = False)

   return q_movies

In [ ]:
gen_df.rename(columns = {'old_name': 'runtime'}, inplace = True)

In [ ]:
personal_recommendations = build_chart(gen_df).head(8)

Input preferred overview
horror
horror
Input shortest duration
20
20
Input longest duration
80
80
Input earliest year
2000
2000
Input latest year
2008
2008


In [ ]:
personal_recommendations

,title,vote_average,vote_count,year,overview,score
0,The Shawshank Redemption,8.7,21840,1994,NaN,8.632212
19000,The Shawshank Redemption,8.7,21840,1994,NaN,8.632212
18800,The Shawshank Redemption,8.7,21840,1994,NaN,8.632212
18820,The Shawshank Redemption,8.7,21840,1994,NaN,8.632212
18840,The Shawshank Redemption,8.7,21840,1994,NaN,8.632212
18860,The Shawshank Redemption,8.7,21840,1994,NaN,8.632212
18880,The Shawshank Redemption,8.7,21840,1994,NaN,8.632212
18900,The Shawshank Redemption,8.7,21840,1994,NaN,8.632212
